In [1]:
!wget ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz -O /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta.gz

--2019-01-30 13:03:30--  ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
           => ‘/Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta.gz’
Resolving ftp.uniprot.org... 141.161.180.197
Connecting to ftp.uniprot.org|141.161.180.197|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/uniprot/current_release/knowledgebase/complete ... done.
==> SIZE uniprot_sprot.fasta.gz ... 88385361
==> PASV ... done.    ==> RETR uniprot_sprot.fasta.gz ... done.
Length: 88385361 (84M) (unauthoritative)

uniprot_sprot.fasta 100%[===================>]  84.29M  6.85MB/s    in 11s     

2019-01-30 13:03:43 (7.44 MB/s) - ‘/Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta.gz’ saved [88385361]



In [2]:
!gunzip -k /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta.gz

In [3]:
!/Users/Shared/bioinformatics/ncbi-blast-2.5.0+/bin/makeblastdb \
-in /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta \
-out /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_db \
-dbtype prot



Building a new DB, current time: 01/30/2019 13:11:04
New DB name:   /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_db
New DB title:  /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 559077 sequences in 55.0913 seconds.


In [4]:
#Confirm Steven's .fa file is the same as Sean's
!diff /Volumes/web/halfshell/bu-git-repos/nb-2017/C_gigas/data/Cg_Giga_cont_AA.fa \
/Volumes/web/scaphapoda/Sean/Rhonda-2016-Oyster-Intermediates/Cg-Giga_cont_AA.fa

In [ ]:
#can sort database before aligning to change the order of entries. We don't care that much about taxanomic group, we car about protein functionality. So it doesn't matter for us if human gets called over mouse or vice-versa. So I am not worrying about changing the max_target_seqs parameter

In [ ]:
!/Users/Shared/bioinformatics/ncbi-blast-2.5.0+/bin/blastp \
-query /Volumes/web/halfshell/bu-git-repos/nb-2017/C_gigas/data/Cg_Giga_cont_AA.fa \
-db /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_db \
-out /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.tab \
-num_threads 8 \
-max_hsps 1 \
-max_target_seqs 1 \
-outfmt 6

In [ ]:
#formatting output so that uniprot accessions are in one column
#first test to see if all uniprot db entries start with the same pattern "sp|"
#This awk command finds every line containing a '>' and prints the first three characters after the carrot
#Awk command literally: if first field contains a carrot, print a substring of the first field string, starting at the second character in the string, that is 3 characters long
# sort the list of substrings and print only unique substrings and their frequencies
#if there is only one substring, we can conclude all uniprot DB entries start with the same pattern "sp|"
!awk '{if($1~/>/)print substr($1,2,3)}' uniprot_swprot2019_01.fasta | sort | uniq -c

In [ ]:
#Next check that there is only 1 pipe in each uniprot DB name after removing the "sp|" pattern
#we tell awk that our file is tab delimited, then print only the sequence name
#then sed to remove the 'sp|' pattern
#then sed to replace all non '|' characters with nothing and awk to count the number of characters remaining in each line
#https://unix.stackexchange.com/questions/18736/how-to-count-the-number-of-a-specific-character-in-each-line
#then sort and unique to see if there are different number of pipes in each line
awk -F"\t" '{if($1~/>/)print $1}' uniprot_swprot2019_01.fasta | sed 's/sp\|//g' | sed 's/[^|]//g' | awk '{print length}' | sort | uniq -c

In [ ]:
#Next check that the uniprot DB does not contain the pattern "|m"
#And that only the C_gigas sequence names contain "|m"
#We want to check this so that we can replace "\m" with ".m"
!grep '\|m' uniprot_swprot2019_01.fasta | head
!grep '\|m' /Volumes/web/halfshell/bu-git-repos/nb-2017/C_gigas/data/Cg_Giga_cont_AA.fa | head


In [ ]:
# now in our BLAST output file, we can replace 'sp|' with nothing, "|m" with ".m", and then finally we can replace the only remaining pipe with tabs

In [ ]:
#Because all uniprot DB entries start with the same pattern, we can replace it with nothing so that each entry starts with the uniprot accession

!sed 's/sp\|//g'  | sed 's/\|m/\.m/g' | tr '|' '\t' 